# House Price Prediction using Linear Regression

House price prediction is a common task in real estate and property markets. It involves estimating the value of a house based on various factors such as the number of bedrooms, the size of the house and other relevant features. Linear regression is a widely used statistical technique that can be applied to predict house prices based on these features.

In this Python code, we will demonstrate how to build a simple linear regression model to predict house prices. We will use a dataset that contains information about different houses, including their features and corresponding prices. The goal is to train a regression model that can accurately estimate house prices based on the provided features.

In [ ]:
import pandas as pd
import numpy as np

from scipy import stats
from src.linear_regression import GDLinearRegression, LinearRegression
from sklearn.linear_model import LinearRegression as SklearnLinearRegression
from src.tester import cross_validate, format_score

## Load the data

The house price dataset contains 18 features from the year 2014 during the months from february to october. The dataset contains the relevant features for predicting the price which are :

1) date : the date of the data for each house
2) price : the actual price for the house
3) bedrooms: the number of bedrooms in the house
4) bathrooms: the number of bathrooms in the house
5) sqft_living: the total living in square feet which is the area that will be heated or cooled or the area where you spend your time
6) sqft_lot: the total lot in square feet which is the land you own according to the boundary lines determined by the city
7) floors: the number of floors in the house
8) waterfront: indicates whether the house has direct access to a natural or man-made waterway such as a lake, river, channel or canal.
9) view: the view the house has
10) condition: a numerical value that rates the condition of the house
11) sqft_above: the total of all living square feet in a home that is above the ground.
12) sqft_basement: the total square feet of the basement
13) yr_built: the year the house was built in
14) yr_renovated: the year the house was renovated in
15) street: the address of the house and which street it is on
16) city: the city the house is in
17) statezip: the zip code for the state the house is in
18) country: the country the house is located in


In [ ]:
df = pd.read_csv("data/house-price-data.csv")
df.head(5)


## Preprocess the data


Encoding city names


In [ ]:
df["city_counts"] = df["city"].map(df["city"].value_counts())


Outlier detection and removal


In [ ]:
df = df[(np.abs(stats.zscore(df["price"])) < 2.9)]
df = df[df["city_counts"] > 100]

Feature and label splitting


In [ ]:
X, y = df.loc[:, df.columns != "price"], df["price"]

Dropping unnecessary columns


In [ ]:
X = X.drop(["date", "street", "statezip", "country", "city"], axis=1, inplace=False)

Normalization


In [ ]:
X = (X - X.min()) / (X.max() - X.min())
y = (y - y.min()) / (y.max() - y.min())

In [ ]:
X.head(5)

### Alternative Preprocessing Techniques

## Initialize models

We built two models for this project. One is a simple linear regression model and the other is a linear regression model optimized using gradient descent. The two models will be compared to scikit-learn's linear regression model.

### Least Squares Linear Regression

The least squares linear regression model is a simple linear regression model that uses the ordinary least squares method to estimate the regression coefficients. The regression coefficients are represented by the Greek letter beta (β). The regression coefficients are used to estimate the target variable (y) using the following equation:


$$y = \beta X$$

In matrix form, the variables are represented in the following way:

$$
y=
\begin{pmatrix}
y_{1}\\
y_{2} \\
\vdots \\
y_{n}
\end{pmatrix},

\beta=
\begin{pmatrix}
\beta_{0}\\
\beta_{1} \\
\beta_{2} \\
\vdots \\
\beta_{p}
\end{pmatrix},
X=
\begin{pmatrix}
1 & x_{1,1} & x_{1,2} & \cdots & x_{1,p} \\
1 & x_{2,1} & X_{2,2} & \cdots & x_{2,p} \\
\vdots  & \vdots  & \vdots  & \ddots & \vdots  \\
1 & x_{n,1} & x_{n,2} & \cdots & x_{n,p} 
\end{pmatrix}
$$

$X^T.X$ is the covariance matrix of the features and $X^T.y$ is the covariance matrix of the features and the target variable. Their dot product is the covariance matrix of the features and the target variable. The inverse of the covariance matrix of the features is multiplied by the covariance matrix of the features and the target variable to obtain the regression coefficients.

$$\beta = (X^TX)^{-1}X^Ty$$

#### Problems with one hot encoding

When using least squares to fit a linear regression model, we get the inverse of the matrix $X^TX$. If the matrix $X^TX$ is not invertible, then the model cannot be fit using least squares. This is a problem when using one hot encoding to encode categorical features. One hot encoding creates a matrix with a column for each category. If there are too many categories, then the matrix $X^TX$ will not be invertible. This is known as the dummy variable trap. To avoid this problem, we can use a technique called one hot encoding with dummy variables. This technique creates a matrix with a column for each category minus one. This ensures that the matrix $X^TX$ is invertible.

### Optimized Linear Regression using Gradient Descent

Optimizing linear regression using gradient descent is about finding the optimal values for the regression coefficients that minimize the cost function by iteratively updating the regression coefficients in the direction of steepest descent. 
It contains hyperparameters such as: 
- Learning Rate: the step size taken in each iteration of gradient descent
- Number of Iterations: the number that determines the maximum number of times gradient descent updates the regression coefficients.
- The threshold: a value that acts as a stopping criterion for the gradient descent algorithm. Once the magnitude of the gradient vector falls   below the threshold, the algorithm terminates.

$$\beta = \beta - \alpha \frac{2}{n}X^T(X\beta - y)$$

In [ ]:
models = {
    "LinearRegression": LinearRegression(),
    "GDLinearRegression": GDLinearRegression(
        learning_rate=0.05, threshold=1e-9, max_iter=2500
    ),
    "SklearnLinearRegression": SklearnLinearRegression(),
}

## Test each model

We cross-validate each model using 5-fold cross-validation and compare the results.

In [ ]:
for name, model in models.items():
    scores = cross_validate(5, model, X, y)
    for k, v in scores.items():
        print(f"Model: {name}\n - {k}: {format_score(v)}")

## Conclusion

Here, the optimized linear regression model performed slightly better than the simple linear regression model. However, it took a longer time to train the optimized linear regression model. However, the model is prone to errors in case the feature matrix is not invertible. This makes the optimized gradient descent model more reliable.